# Pan-tilt Control And LED Control 
## Pan-tilt Control
### Introduction

The product (PT Version only) with pan-tilt has two servos, that is, pan servo and tilt servo, and the rotation range is ±180° (360° in total). The tilt servo controls the horizontal rotation, and the range is -45°~90° (135° in total)

For products without a pan-tilt mechanism, users can expand this function by adding a pan-tilt mechanism to the product platform themselves. 

In [1]:
import serial, json

ser = serial.Serial("/dev/ttyTHS1", 115200, timeout=1)

while True:
    line = ser.readline()
    if not line:
        print("...no data")
        continue
    print("RAW:", line)
    try:
        data = json.loads(line.decode("utf-8"))
        print("JSON:", data)
    except Exception as e:
        print("PARSE ERR:", e)


RAW: b'0,"odl":-1,"odr":-1,"v":1179}\r\n'
PARSE ERR: Extra data: line 1 column 2 (char 1)
RAW: b'{"T":1001,"L":0,"R":0,"ax":-16,"ay":-296,"az":8526,"gx":-26,"gy":28,"gz":4,"mx":12,"my":60,"mz":-287,"odl":-1,"odr":-1,"v":1179}\r\n'
JSON: {'T': 1001, 'L': 0, 'R': 0, 'ax': -16, 'ay': -296, 'az': 8526, 'gx': -26, 'gy': 28, 'gz': 4, 'mx': 12, 'my': 60, 'mz': -287, 'odl': -1, 'odr': -1, 'v': 1179}
RAW: b'{"T":1001,L":0,"R":0,"ax":-16,"ay":-296,"az":8526,"gx":-26,"gy":28,"gz":4,"mx":12,"my":60,"mz":-287,"odl":-1,"odr":-1,"v":1179}\r\n'
PARSE ERR: Expecting property name enclosed in double quotes: line 1 column 11 (char 10)
RAW: b'{"T":1001,"L":0,"R":0,"ax":-44,"ay":-306,"az":8524,"gx":-20,"gy":30,"gz":0,"mx":43,"my":60,"mz":-247,"odl":-1,"odr":-1,"v":1179}\r\n'
JSON: {'T': 1001, 'L': 0, 'R': 0, 'ax': -44, 'ay': -306, 'az': 8524, 'gx': -20, 'gy': 30, 'gz': 0, 'mx': 43, 'my': 60, 'mz': -247, 'odl': -1, 'odr': -1, 'v': 1179}
RAW: b'{"T":1001,L":0,"R":0,"ax":-42,"ay":-296,"az":8566,"gx":-23,"gy":

SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

In [2]:
# Input the library for base control
from base_ctrl import BaseController

base = BaseController('/dev/ttyTHS1', 115200)

In the code block above, we import and instantiate the library for base control. Next, we control the movement of the pan-tilt by changing the angles of the pan and tilt servos.

Modify the values in the following code:
- input_x: Angle of the pan servo, within the range of ±180° (total range of 360°).
- input_y: Angle of the tilt servo, within the range of -45° to 90° (total range of 135°). 
- input_speed: Speed of the pan-tilt movement. When the speed value is 0, the movement is at its fastest. 
- input_acc: Acceleration at the start and end of the pan-tilt movement. Smaller values result in smoother acceleration and deceleration. When the acceleration value is 0, the maximum acceleration is applied.

Run the code below to observe the pan-tilt move 45° to the right and upward before stopping.

In [6]:
input_x = 0
input_y = 0
input_speed = 0
input_acc = 0

base.gimbal_ctrl(input_x, input_y, input_speed, input_acc)

In addition to controlling the pan-tilt movement by changing the angles of the two servos, you can also directly control its continuous movement.

Modify the values in the code below:

- input_x: the rotation mode of the pan servo. When the value is -1, it represents continuous rotation to the left (clockwise); when the value is 1, it represents continuous rotation to the right (counterclockwise); when the value is 0, it indicates stopping the rotation.
- input_y: the rotation mode of the tilt servo. When the value is -1, it represents continuous downward rotation (tilting down); when the value is 1, it represents continuous upward rotation (tilting up); when the value is 0, it indicates stopping the rotation.
- input_speed: speed of the pan-tilt movement. 

If both input_x and input_y are set to 2, the pan-tilt will automatically return to its middle position.

Run the code below, and the pan-tilt will move to the left until it reaches 180° and then stop.

In [21]:
input_x = -1
input_y = 0
input_speed = 0

base.gimbal_base_ctrl(input_x, input_y, input_speed)

Run the following code, the pan-tilt will move upward until it reaches 90° and then stop. 

In [22]:
input_x = 0
input_y = 1
input_speed = 0

base.gimbal_base_ctrl(input_x, input_y, input_speed)

To make the pan-tilt return to its middle position, run the following code:ter)

In [23]:
input_x = 2
input_y = 2
input_speed = 0

base.gimbal_base_ctrl(input_x, input_y, input_speed)

## LED Control
### Introduction
The WAVE ROVER and UGV series products feature a drive board integrated with two 12V switches (the actual maximum voltage varies with the battery voltage). These switches are controlled by ESP32's IO4 and IO5 pins via MOS tubes. Each switch has two ports, totaling 4x 12V switch control ports. By default, IO4 controls the chassis LED (WAVE ROVER does not have a chassis LED), and IO5 controls the LED. You can control the switching of these two switches and adjust the voltage level by sending the corresponding commands to the slave computer. However, due to the inherent delay in MOSFET control, there may not be a linear relationship between the PWM output from the ESP32's IO and the actual voltage output.

For products without LEDs, you can expand the 12.6V withstand LED on these two 12V switches (in general, 12V withstand is also acceptable for safety and battery protection, the product's UPS will not charge the battery above 12V). You can also expand other peripherals on the remaining switch control interfaces, such as a 12V withstand water gun gearbox, which can be directly connected to the interface controlled by IO5 to achieve automatic aiming and shooting functionality.

To run the code within the code block, you can select the desired code block and then press Ctrl+Enter to run the program. If you are using JupyterLab on a mobile device or tablet, you can press the play button above the code block to run it.

In the code block above, we imported and instantiated the library for controlling the chassis. Next, we will control the switch of interface IO4. The variable IO4_PWM represents the PWM output of ESP32's IO4 pin, ranging from 0 to 255. When this variable is 0, the switch controlled by IO4 is turned off; when this variable is 255, the voltage output of the switch controlled by IO4 is close to the UPS BAT voltage (the current voltage of the three lithium batteries connected in series inside the UPS).

Run the following code block to turn on the switch controlled by IO4 (turn on the chassis LED). Note: RaspRover does not have a chassis LED, so running the following code block will not change anything. You need to run the next code block to turn on the LED, which is located on the camera pan-tilt. If the product does not come with a camera pan-tilt, there are no LED lights (Only available on the PT version).

In [9]:
IO4_PWM = 255
IO5_PWM = 0

base.lights_ctrl(IO4_PWM, IO5_PWM)

To turn on the switch controlled by interface IO5 (turn on the pan-tilt LED), run the following code block:
Note: If the product does not come with a camera pan-tilt, there are no LED lights.

In [13]:
IO4_PWM = 255
IO5_PWM = 255

base.lights_ctrl(IO4_PWM, IO5_PWM)

If your product comes with LED, they should all be tilted up by now. You can run the following code block to decrease the brightness of the LED lights.

In [14]:
IO4_PWM = 64
IO5_PWM = 64

base.lights_ctrl(IO4_PWM, IO5_PWM)

Finally, run the following code block to turn off the LEDs.

In [15]:
base.lights_ctrl(0, 0)

Run a code block here that integrates the pan-tilt functionality:

In [17]:
import time
base.gimbal_ctrl(0, 0, 0, 0)
base.lights_ctrl(255, 255)
time.sleep(1)
base.gimbal_ctrl(45, 0, 0, 0)
base.lights_ctrl(0, 0)
time.sleep(1)
base.gimbal_ctrl(-45, 0, 0, 0)
base.lights_ctrl(255, 255)
time.sleep(1)
base.gimbal_ctrl(0, 90, 0, 0)
base.lights_ctrl(0, 0)
time.sleep(1)
base.gimbal_ctrl(0, 0, 0, 0)